<a href="https://colab.research.google.com/github/vblagoje/notebooks/blob/main/haystack2x-demos/haystack_rag_serperdev_demo_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introduction

This notebook provides a detailed guide on leveraging Haystack 2.x and [serper.dev](https://serper.dev/) for implementing an OpenAPI service-based Retriever-Augmented Generation (RAG) workflow. It outlines an approach for processing user queries by integrating web search results into the context of Large Language Models (LLMs), enhancing their response generation.

The pipeline starts by fetching the OpenAPI specification for serper.dev and converting it into OpenAI function-calling definitions. serper.dev is then used as an OpenAPI service to retrieve relevant information from the web based on the user's input. The pipeline incorporates these search results, along with system and user prompts, into the LLM to enhance response generation.

The notebook will guide you through installing the necessary libraries, selecting an LLM provider, and constructing a Haystack 2.0 pipeline that orchestrates function calling, service requests, and LLM response generation.

Note: To run this pipeline, a [serper.dev](https://serper.dev/) account is required. Signing up is quick and provides you with 2,500 free queries—no credit card needed.

## 1. Setup

Let's install necessary libraries and import key modules to build the foundation for the subsequent steps.

In [18]:
!pip install -q openapi-service-client git+https://github.com/deepset-ai/haystack.git@integrate_openapi_service_client

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
import getpass
import os
import json
import requests

from typing import Dict, Any, List
from haystack import Pipeline
from haystack.components.generators.utils import print_streaming_chunk
from haystack.components.converters import OpenAPIServiceToFunctions, OutputAdapter
from haystack.components.generators.chat import OpenAIChatGenerator
from haystack.components.connectors import OpenAPIServiceConnector
from haystack.components.fetchers import LinkContentFetcher
from haystack.dataclasses import ChatMessage, ByteStream
from haystack.utils import Secret

## 2.  Enter the API keys for LLM provider and serper.dev

In [19]:
# Enable support for multiple LLM providers and simplify switching between them

llm_providers = {"fireworks": {"api_base_url": "https://api.fireworks.ai/inference/v1",
                               "text_model": "accounts/fireworks/models/mixtral-8x7b-instruct",
                               "fc_model": "accounts/fireworks/models/firefunction-v1"},
                 "openai": {"api_base_url": "https://api.openai.com/v1",
                            "text_model": "gpt-4-turbo-preview",
                            "fc_model": "gpt-3.5-turbo"}
                 }

# change this value to another provider defined above - if needed
selected_provider = "openai"

In [20]:
llm_api_key = getpass.getpass(f"Enter LLM api key for {selected_provider}:")
serper_dev_key = getpass.getpass("Enter serperdev api key:")


Enter LLM api key for openai:··········
Enter serperdev api key:··········


## 3. Build our RAG serper.dev pipeline



In [21]:
# An OutputAdapter filter we'll use to setup function calling
def prepare_fc_params(openai_functions_schema: Dict[str, Any]) -> Dict[str, Any]:
    return {"tools": [openai_functions_schema]}

pipe = Pipeline()
pipe.add_component("spec_to_functions", OpenAPIServiceToFunctions())
pipe.add_component("functions_llm", OpenAIChatGenerator(api_key=Secret.from_token(llm_api_key),
                                                        model=llm_providers[selected_provider]["fc_model"],
                                                        api_base_url=llm_providers[selected_provider]["api_base_url"]))
pipe.add_component("openapi_container", OpenAPIServiceConnector())
pipe.add_component("prepare_fc_adapter", OutputAdapter("{{functions[0] | prepare_fc}}", Dict[str, Any], {"prepare_fc": prepare_fc_params}))
pipe.add_component("openapi_spec_adapter", OutputAdapter("{{specs[0]}}", Dict[str, Any]))
pipe.add_component("final_prompt_adapter", OutputAdapter("{{system_message + service_response}}", List[ChatMessage]))
pipe.add_component("llm", OpenAIChatGenerator(api_key=Secret.from_token(llm_api_key),
                                              api_base_url=llm_providers[selected_provider]["api_base_url"],
                                              model=llm_providers[selected_provider]["text_model"],
                                              generation_kwargs={"max_tokens": 1024},
                                              streaming_callback=print_streaming_chunk))

pipe.connect("spec_to_functions.functions", "prepare_fc_adapter.functions")
pipe.connect("spec_to_functions.openapi_specs", "openapi_spec_adapter.specs")
pipe.connect("functions_llm.replies", "openapi_container.messages")
pipe.connect("prepare_fc_adapter", "functions_llm.generation_kwargs")
pipe.connect("openapi_spec_adapter", "openapi_container.service_openapi_spec")
pipe.connect("openapi_container.service_response", "final_prompt_adapter.service_response")
pipe.connect("final_prompt_adapter", "llm.messages")


🚅 Components
  - spec_to_functions: OpenAPIServiceToFunctions
  - functions_llm: OpenAIChatGenerator
  - openapi_container: OpenAPIServiceConnector
  - prepare_fc_adapter: OutputAdapter
  - openapi_spec_adapter: OutputAdapter
  - final_prompt_adapter: OutputAdapter
  - llm: OpenAIChatGenerator
🛤️ Connections
  - spec_to_functions.functions -> prepare_fc_adapter.functions (List[Dict[str, Any]])
  - spec_to_functions.openapi_specs -> openapi_spec_adapter.specs (List[Dict[str, Any]])
  - functions_llm.replies -> openapi_container.messages (List[ChatMessage])
  - openapi_container.service_response -> final_prompt_adapter.service_response (Dict[str, Any])
  - prepare_fc_adapter.output -> functions_llm.generation_kwargs (Dict[str, Any])
  - openapi_spec_adapter.output -> openapi_container.service_openapi_spec (Dict[str, Any])
  - final_prompt_adapter.output -> llm.messages (List[ChatMessage])

As you can see in the pipeline graph above, for a given serper.dev query, our pipeline follows these steps:

1. **Fetch OpenAPI Spec**: Retrieve the OpenAPI specification for serper.dev and convert it into OpenAI function-calling definitions.

2. **Determine Parameters**: Use a function-calling model to identify the necessary parameters for the serper.dev service.

3. **Query serper.dev**: Dispatch the request to serper.dev with the determined parameters and gather the responses.

4. **Compile Results**: Organize and format the search results from serper.dev.

5. **Generate Response**: Combine the system prompt, user query, and compiled search results, then pass them to LLM to generate the final response.

In [22]:
system_prompt = requests.get("https://bit.ly/serper_dev_system").text
serper_spec = requests.get("https://bit.ly/serper_dev_spec").text

In [23]:
query = "Why did Elon Musk sue OpenAI?"

result = pipe.run(data={"functions_llm": {"messages":[ChatMessage.from_system("Only do function calling"), ChatMessage.from_user(query)]},
                        "openapi_container": {"service_credentials": serper_dev_key},
                        "spec_to_functions": {"sources": [ByteStream.from_string(serper_spec)]},
                        "final_prompt_adapter": {"system_message": [ChatMessage.from_system(system_prompt)]}})

Elon Musk sued OpenAI, accusing the company of breaching its founding agreement and diverging from its original, nonprofit mission by reserving some of its most advanced artificial intelligence technology for private customers.